In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [3]:
store_files = [
    "synthetic_data/RCT_0_5.h5",
    "synthetic_data/RCT_0_05.h5",
    "synthetic_data/e_X.h5",
    "synthetic_data/e_X_U.h5",
    "synthetic_data/e_X_info_censor.h5",
    "synthetic_data/e_X_U_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            continue
    experiment_setups[base_name] = scenario_dict

In [4]:
experiment_setups['RCT_0_5']['scenario_1']['dataset'].head()

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613


In [5]:
experiment_repeat_setups = pd.read_csv("synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# Run Experiments

In [6]:
from models_causal_impute.meta_learners import TLearner, SLearner, XLearner
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [7]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list):
    split_results = {}

    for col in random_idx_col_list:
        random_idx = experiment_repeat_setups[col].values
        test_ids = random_idx[-5000:]
        train_ids = random_idx[:-5000]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        CATE_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[col] = (X_train, W_train, Y_train, X_test, W_test, Y_test, CATE_test_true)

    return split_results

In [8]:
random_idx_col_list = ["random_idx0"]
failure_times_grid_size = 500
output_pickle_path = "causal_impute_s_learner_random_idx0_train_45000.pkl"

base_regressors = ['ridge', 'lasso', 'rf', 'gbr', 'xgb']

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):
    for scenario_key in tqdm(setup_dict, desc=f"{setup_name} Scenarios"):
        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list)

        if "result" not in experiment_setups[setup_name][scenario_key]:
            experiment_setups[setup_name][scenario_key]["result"] = {}


        # For each base model, we will run the SLearner
        for base_model in tqdm(base_regressors, desc="Base Models", leave=False):
            # print(f"Running {base_model} for {setup_name} - {scenario_key}")
            
            # Store placeholder for later population
            experiment_setups[setup_name][scenario_key]["result"][base_model] = {}

            
            start_time = time.time()

            for col in random_idx_col_list:
                X_train, W_train, Y_train, X_test, W_test, Y_test, CATE_test_true = split_dict[col]

                ##################################
                # TODO: Imputation Step
                Y_train = Y_train[:, 0]  # observed_time
                Y_test  = Y_test[:, 0]   # observed_time
                ##################################
                
                
                learner = SLearner(base_model_name=base_model)
                learner.fit(X_train, W_train, Y_train)
                mse_test, CATE_test_pred = learner.evaluate(X_test, CATE_test_true)

                experiment_setups[setup_name][scenario_key]["result"][base_model][col] = {
                    "CATE_true": CATE_test_true,
                    "CATE_Pred": CATE_test_pred,
                    "CATE_MSE": mse_test
                }

            end_time = time.time()

            experiment_setups[setup_name][scenario_key]["result"][base_model]["average"] = {
                "mean_CATE_MSE": np.mean([experiment_setups[setup_name][scenario_key]["result"][base_model][col]["CATE_MSE"]
                                        for col in random_idx_col_list]),
                "std_CATE_MSE":  np.std( [experiment_setups[setup_name][scenario_key]["result"][base_model][col]["CATE_MSE"]
                                        for col in random_idx_col_list]),
                "runtime": (end_time - start_time) / len(random_idx_col_list)
            }

            # Save progress to disk
            # with open(output_pickle_path, "wb") as f:
                # pickle.dump(experiment_setups, f)
            
            # break
        # break
    # break
            


Experiment Setups:   0%|          | 0/6 [00:00<?, ?it/s]























































Experiment Setups:  17%|█▋        | 1/6 [11:08<55:41, 668.26s/it]























































Experiment Setups:  33%|███▎      | 2/6 [22:21<44:44, 671.09s/it]























































Experiment Setups:  50%|█████     | 3/6 [31:58<31:25, 628.39s/it]























































Experiment Setups:  67%|██████▋   | 4/6 [42:26<20:55, 627.96s/it]


































Experiment Setups:  83%|████████▎ | 5/6 [49:18<09:10, 550.33s/it]


































Experiment Setups: 100%|██████████| 6/6 [56:49<00:00, 568.19s/it]


In [9]:
def summarize_experiment_results(experiment_setups, base_regressors):
    records = []

    for setup_name, setup_dict in experiment_setups.items():
        for scenario_key in setup_dict:
            row = {
                ("setup_name", ""): setup_name,
                ("scenario_key", ""): scenario_key
            }

            for base_model in base_regressors:
                avg_result = setup_dict[scenario_key].get("result", {}).get(base_model, {}).get("average", {})
                mean_mse = avg_result.get("mean_CATE_MSE", np.nan)
                std_mse = avg_result.get("std_CATE_MSE", np.nan)
                runtime = avg_result.get("runtime", np.nan)

                row[(base_model, "mean_CATE_mse")] = round(mean_mse, 4) if not pd.isna(mean_mse) else np.nan
                # row[(base_model, "std_CATE_mse")] = round(std_mse, 4) if not pd.isna(std_mse) else np.nan
                row[(base_model, "runtime [s]")] = round(runtime, 4) if not pd.isna(runtime) else np.nan

            records.append(row)

    df = pd.DataFrame.from_records(records)
    df.columns = pd.MultiIndex.from_tuples(df.columns)
    return df


In [10]:
print("S-Learner Results")
summary_df = summarize_experiment_results(experiment_setups, base_regressors)
summary_df

S-Learner Results


setup_name scenario_key         ridge                     lasso  \
                                   mean_CATE_mse runtime [s] mean_CATE_mse   
0             RCT_0_5   scenario_1        0.7019      1.9046        0.7021   
1             RCT_0_5   scenario_2       15.9475      0.2720       15.9481   
2             RCT_0_5   scenario_3       14.6995      0.1854       14.6996   
3             RCT_0_5   scenario_5       14.7865      0.1896       14.7871   
4             RCT_0_5   scenario_6       15.0894      0.1699       15.0919   
5             RCT_0_5   scenario_8       17.2537      0.2620       17.2803   
6             RCT_0_5   scenario_9       43.1499      0.1671       43.1462   
7             RCT_0_5  scenario_10       15.9384      0.2357       15.9389   
8            RCT_0_05   scenario_1        0.7002      0.1632        0.7008   
9            RCT_0_05   scenario_2       15.9492      0.2463       15.9497   
10           RCT_0_05   scenario_3       14.6949      0.1749       14.7026   
11           RCT_0_05   scenario_5       14.7727      0.4351       14.7748   
12           RCT_0_05   scenario_6       15.0412      0.2003       15.0463   
13           RCT_0_05   scenario_8       17.2628      0.3784       17.3272   
14           RCT_0_05   scenario_9       43.1476      0.1637       43.1393   
15           RCT_0_05  scenario_10       15.9447      0.1968       15.9461   
16                e_X   scenario_1        0.7044      0.1731        0.7049   
17                e_X   scenario_2       15.9451      0.3291       15.9456   
18                e_X   scenario_3       14.7097      0.1629       14.7100   
19                e_X   scenario_5       14.8313      0.1893       14.8325   
20                e_X   scenario_6       15.0897      0.3843       15.0933   
21                e_X   scenario_8       17.2953      0.1629       17.3292   
22                e_X   scenario_9       43.1526      0.2350       43.1498   
23                e_X  scenario_10       15.9485      0.1722       15.9491   
24              e_X_U   scenario_1        0.7021      0.4734        0.7024   
25              e_X_U   scenario_2       15.9490      0.1892       15.9497   
26              e_X_U   scenario_3       14.7031      0.1636       14.7033   
27              e_X_U   scenario_5       14.8256      0.1773       14.8265   
28              e_X_U   scenario_6       15.0958      0.1736       15.0986   
29              e_X_U   scenario_8       17.2765      0.3347       17.3069   
30              e_X_U   scenario_9       43.1437      0.1745       43.1404   
31              e_X_U  scenario_10       15.9441      0.3966       15.9446   
32    e_X_info_censor   scenario_1        0.6994      0.1646        0.6997   
33    e_X_info_censor   scenario_2       15.9451      0.1660       15.9456   
34    e_X_info_censor   scenario_3       15.4284      0.1763       15.4239   
35    e_X_info_censor   scenario_8       17.4423      0.4355       17.4382   
36    e_X_info_censor   scenario_9       42.9373      0.1733       42.9346   
37  e_X_U_info_censor   scenario_1        0.6976      0.2941        0.6978   
38  e_X_U_info_censor   scenario_2       15.9495      0.1637       15.9497   
39  e_X_U_info_censor   scenario_3       15.3927      0.4224       15.3893   
40  e_X_U_info_censor   scenario_8       17.5113      0.3057       17.5080   
41  e_X_U_info_censor   scenario_9       42.8722      0.1811       42.8694   

                          rf                       gbr              \
   runtime [s] mean_CATE_mse runtime [s] mean_CATE_mse runtime [s]   
0       0.3593        0.6904     32.5980        0.6931     26.3947   
1       0.2932       15.8271     33.1710       15.8429     29.2579   
2       1.1612       14.5174     31.7963       14.5246     28.8101   
3       0.3863       14.6692     33.2921       14.6911     27.0843   
4       0.3329       15.2391     40.5589       15.0949     29.3134   
5       0.3491       17.2917     41.1072       17.3104     26.9444   
6       0.2264    